<a href="https://colab.research.google.com/github/Chiebukar/Retinanet_logo_detection/blob/main/retinanet_logo_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# # mount google drive
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
import os

In [ ]:
# installing retinanet
!git clone https://github.com/fizyr/keras-retinanet
os.chdir('keras-retinanet')
!python setup.py install

In [ ]:
# # import required libraries
# import keras
# import cv2
# import keras_retinanet

In [ ]:
# # checking retinanent-train command
# !retinanet-train

In [ ]:
os.chdir('/content/drive/MyDrive/Retinanet/Retinanet_logo_detection')  # redirect worrking dir to base folder

# indicate bash command
%%bash 
mkdir -p config   # create config folder
touch config/__init__.py # create __init__.py  file
touch config/logos_config.py # create logos_config.py  file

In [ ]:
%%writefile config/logos_config.py

import os

# initiaize the base path to the  dataset
BASE_PATH = 'logos'

# build the path to the annotations and input images
ANNOT_PATH = os.path.sep.join([BASE_PATH, 'annotations'])
IMAGES_PATH = os.path.sep.join([BASE_PATH, 'images'])
                              
# build the path to the training and testing .txt files
TRAIN_TXT = os.path.sep.join([BASE_PATH, "train.txt"])
TEST_TXT = os.path.sep.join([BASE_PATH, "test.txt"])

# build the path to the training and test .csv files
TRAIN_CSV = os.path.sep.join([BASE_PATH, "retinanet_train.csv"])
TEST_CSV = os.path.sep.join([BASE_PATH, "retinanet_test.csv"])

# build the path to the output classes CSV file
CLASSES_CSV = os.path.sep.join([BASE_PATH, "retinanet_classes.csv"])                            

Overwriting config/logos_config.py


In [ ]:
# # download and copy the Resnet50_Coco file the logos directory
# !wget https://github.com/fizyr/keras-retinanet/releases/download/0.5.1/resnet50_coco_best_v2.1.0.h5
# !mv resnet50_coco_best_v2.1.0.h5 /content/drive/MyDrive/Retinanet/Retinanet_logo_detection/logos

In [ ]:
%%writefile build_logos.py

# import the necessary packages
from config import logos_config as config
from bs4 import BeautifulSoup
import os

# initialize the set of classes we have encountered so far
CLASSES = set()

# create the list of datasets to build
datasets = [
	("train", config.TRAIN_TXT, config.TRAIN_CSV),
	("test", config.TEST_TXT, config.TEST_CSV),
]

# loop over the datasets
for (dType, inputTxt, outputCSV) in datasets:
	# load the contents of the input data split
	print("[INFO] starting '{}' set...".format(dType))
	imageIDs = open(inputTxt).read().strip().split("\n")
	print("[INFO] {} total images in '{}' set".format(
		len(imageIDs), dType))

	# open the output CSV file
	csv = open(outputCSV, "w")

	# loop over the image IDs
	for imageID in imageIDs:
		# build the path to the image path and annotation file
		imagePath = "{}.jpg".format(os.path.sep.join([
			config.IMAGES_PATH, imageID]))
		annotPath = "{}.xml".format(os.path.sep.join([
			config.ANNOT_PATH, imageID]))

		# load the annotation file, build the soup, and initialize
		# the set of coordinates for this particular image
		contents = open(annotPath).read()
		soup = BeautifulSoup(contents, "html.parser")
		coords = set()

		# extract the image dimensions
		w = int(soup.find("width").string)
		h = int(soup.find("height").string)

		# loop over all `object` elements
		for o in soup.find_all("object"):
			# extract the label and bounding box coordinates
			label = o.find("name").string
			xMin = int(o.find("xmin").string)
			yMin = int(o.find("ymin").string)
			xMax = int(o.find("xmax").string)
			yMax = int(o.find("ymax").string)

			# truncate any bounding box coordinates that may fall
			# outside the boundaries of the image
			xMin = max(0, xMin)
			yMin = max(0, yMin)
			xMax = min(w, xMax)
			yMax = min(h, yMax)

			# build a (hashable) tuple from the coordinates
			coord = (xMin, yMin, xMax, yMax)

			# if the coordinates already exist in our `coords` set,
			# ignore the annotation (this is a peculiarity of the
			# logos dataset)
			if coord in coords:
				continue

			# write the image path, bounding box coordinates, and
			# label to the output CSV file
			row = [os.path.abspath(imagePath), str(xMin), str(yMin),
				str(xMax), str(yMax), label]
			csv.write("{}\n".format(",".join(row)))

			# add the bounding box coordinates to our set and update
			# the set of class labels
			coords.add(coord)
			CLASSES.add(label)

	# close the output CSV file
	csv.close()

# write the classes to file
print("[INFO] writing classes...")
csv = open(config.CLASSES_CSV, "w")
rows = [",".join([c, str(i)]) for (i, c) in enumerate(CLASSES)]
csv.write("\n".join(rows))
csv.close()

Overwriting build_logos.py


In [ ]:
# run build_logos.py
!python build_logos.py

[INFO] starting 'train' set...
[INFO] 748 total images in 'train' set
[INFO] starting 'test' set...
[INFO] 187 total images in 'test' set
[INFO] writing classes...


In [ ]:
!wc -l logos/retinanet_train.csv

1393 logos/retinanet_train.csv


In [ ]:
1393//2

696

In [ ]:
!mkdir logos/snapshots
!retinanet-train --batch-size 2 --steps 374 --epochs 28 \
--weights logos/resnet50_coco_best_v2.1.0.h5 \
--snapshot-path logos/snapshots \
csv logos/retinanet_train.csv logos/retinanet_classes.csv

In [ ]:
# export model
!retinanet-convert-model logos/snapshots/resnet50_csv_28.h5 output.h5

2021-11-14 18:48:55.321234: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


In [ ]:
!retinanet-evaluate csv logos/retinanet_test.csv \
logos/retinanet_classes.csv output.h5

Loading model, this may take a second...
2021-11-14 18:50:29.438913: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Running network: 100% (187 of 187) |######| Elapsed Time: 0:19:08 Time:  0:19:08
Parsing annotations: 100% (187 of 187) |##| Elapsed Time: 0:00:00 Time:  0:00:00
16 instances of class FC_Barcelona with average precision: 0.6631
16 instances of class Audi-Pict with average precision: 0.5307
5 instances of class Audi-Text with average precision: 0.0500
13 instances of class Facebook-Text with average precision: 0.4491
22 instances of class Allianz-Text with average precision: 0.6641
25 instances of class Adidas-Pict with average precision: 0.4238
23 instances of class FC_Bayern_Munchen with average precision: 0.7873
4 instances of class Atletico_Madrid with average precision: 1.0000
13 instances of class Amazon with average precision: 0.7981
36 instances of class eBay with average precisi

In [ ]:
!touch predict.py 
%%writefile predict.py

# import the necessary packages
from keras_retinanet.utils.image import preprocess_image
from keras_retinanet.utils.image import read_image_bgr
from keras_retinanet.utils.image import resize_image
from keras_retinanet import models
import numpy as np
import argparse
import cv2

# construct the argument parse and parse the arguments
ap = argparse.ArgumentParser()
ap.add_argument("-m", "--model", required=True,
	help="path to pre-trained model")
ap.add_argument("-l", "--labels", required=True,
	help="path to class labels")
ap.add_argument("-i", "--image", required=True,
	help="path to input image")
ap.add_argument("-c", "--confidence", type=float, default=0.5,
	help="minimum probability to filter weak detections")
args = vars(ap.parse_args())

# load the class label mappings
LABELS = open(args["labels"]).read().strip().split("\n")
LABELS = {int(L.split(",")[1]): L.split(",")[0] for L in LABELS}

# load the model from disk
model = models.load_model(args["model"], backbone_name="resnet50")

# load the input image (in BGR order), clone it, and preprocess it
image = read_image_bgr(args["image"])
output = image.copy()
image = preprocess_image(image)
(image, scale) = resize_image(image)
image = np.expand_dims(image, axis=0)

# detect objects in the input image and correct for the image scale
(boxes, scores, labels) = model.predict_on_batch(image)
boxes /= scale

# loop over the detections
for (box, score, label) in zip(boxes[0], scores[0], labels[0]):
	# filter out weak detections
	if score < args["confidence"]:
		continue

	# convert the bounding box coordinates from floats to integers
	box = box.astype("int")

	# build the label and draw the label + bounding box on the output
	# image
	label = "{}: {:.2f}".format(LABELS[label], score)
	cv2.rectangle(output, (box[0], box[1]), (box[2], box[3]),
		(0, 255, 0), 2)
	cv2.putText(output, label, (box[0], box[1] - 10),
		cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

# show the output image
cv2.imshow("Output", output)
cv2.waitKey(0)

SyntaxError: ignored

In [ ]:
!python predict.py --model output.h5 --labels logos/retinanet_classes.csv \
--image logos/images/786414.jpg --confidence 0.5